# API design - lessons learned

- Sometimes, one goodidea 
- Is over-powered by another conflicting goal

## Environmental consideration
- Does the API need to conform to a spec? If so, your hands are tied
    - A good way
    - Improves learnability
    - Spec usually comes with tests (decimal)

- Is there some well-known mataphor or other (external) expectation about what the API whould look like? 
    - Sets, for example (No learning curve)
    - Study approaches in Java, c, etc
    
- Will the API be published broadly enought that you can't easily change it? 
    - hard to change down in the road
    - Publish minimal API vs full api that as useful as possible
    
    
## Designing array indicies to integers
- We wanted `a[3L]` to work like `a[3]` and didn't mind `a[3.0]`
- But `a[3.4]` working like `a[3]` is right out 
- new api: `__index__`

- Principle
    - when need to force multi input types into single type, need to tightened up and unwanted type need get caught
    
## Designing classes for subclassing
- We wanted to override `dict.__setitem__` and have everything work
- But we don't want to break invariants

- At the time when designing class, no need for subclassing
- Class design not done until considering subclasses
    - `super.__init__`
    - protect internal methods
    - `__name`
    
- e.g., when subclasing `dict`, need to override every methods that's potentially change functionality

- When designing own class, consider invariants

- provide hooks instead of hard wiring class: monkey patch
    - `dict.__getitem__` calls `__missing__()`


## OCP
```Python
class D:
    def __setitem__(self, b, c):
        pass
    def update(self, items):
        for k, v in items:
            self[k] = v
class E(D):
    def __setitem__(self, b, c): pass #changed
    # When E use update, it will use new setitem 
    # behavior change
    

    
class D:
    __local__setitem__ = __setitem__ 
    ...
    self.__local__setitem__(k, v)
```

## Recursive `__repr__`
- we want to write: 
```Python
@recursive_repr
def __repr__(*args):
    ...
```
and get output like `{"a": ...}`

- But we got
```Python
@recursive_repr
def __repr__(*args):
    ...
```

- Why the decorator has `()`?
    - The output `...` in `{'a':...}` is hard-wired
    - ` @reprlib.recursive_repr(fillvalue="...") `
- To provide an __hook__: One or two arguments plus lots of default 
    - So that people can control behavior
    
- Principle
    - hardwiring choises will severly limit your options for users to provide optimal arguments and flexiability (if you don't control the client code)
    - but IF I can control the client code, the better option is to supply without the control line (simplest working code)

In [5]:
d = {}
d['k'] = 3
d['e'] = d
d

{'k': 3, 'e': {...}}

- principle
Behaviour should be recognizable so person can anticipate what it going to do, not creating confusion

- principle
    - be nice, provocable, forgiving
    - when a guy starts gussing what the other guy is doing, the behaviour became unrecognizable; others can't recognize its nice so start behaving evil

## Conflicting constructor requirements
- We want to write `itertools.chain(a, b, c)` and `SomeNT(a, b, c)`
- But also need `itertools.chain(iterable)` and `SomeNT(iterable)`

- When there are conflicting requirements for a constructor, the preferred solution is to serve each need with its own `classmethod`
- Combining multiple signatures into one constructor hasn't worked out as well

In [8]:
##
a = list(range(10))
a[:2]

[0, 1]